Sources

#### Projet End-to-End (Pas NHL API)
https://medium.com/coinmonks/4-718-using-machine-learning-to-bet-on-the-nhl-25d16649cd52

#### Projet Live Feed (NHL API)
##### Utilise le live feed pour shot sur gardien
https://towardsdatascience.com/nhl-analytics-with-python-6390c5d3206d

#### Docu NHL API
https://github.com/dword4/nhlapi

Ancienne Doc:
https://webcache.googleusercontent.com/search?q=cache:ILYmzDuxW70J:https://github.com/dword4/nhlapi+&cd=1&hl=en&ct=clnk&gl=ca

In [1]:
# Check Python Version
from platform import python_version
import pandas as pd
import requests  
from pandas.io.json import json_normalize
import datetime

pd.set_option('display.max_columns', None)

print(python_version())

3.6.5


### import game data

In [2]:
start_date = datetime.date(2007, 10, 1)
end_date = datetime.date(2019, 3, 31)
delta = datetime.timedelta(days=1)

data_list = []

while start_date <= end_date:
    
    r = requests.get(url='https://statsapi.web.nhl.com/api/v1/schedule?date=' + start_date.strftime("%Y-%m-%d"))
    data = r.json()
    
    df = json_normalize(data = data['dates'], record_path='games', meta=['date', 'totalGames'])
    
    data_list.append(df)
        
    start_date += delta

In [4]:
df = pd.concat(data_list, sort='False', ignore_index=True)

df.to_csv('../../data/games.csv')

### Import teams data

In [5]:
url = 'https://statsapi.web.nhl.com/api/v1/teams'
r = requests.get(url)
json_data = r.json()
teams = json_normalize(json_data['teams'], sep = "_")

teams.to_csv('../../data/teams.csv')